<a href="https://colab.research.google.com/github/ogut77/LogisticRegression/blob/master/LGBM_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

from lightgbm import LGBMClassifier
import gc

# CSV Data Loading
data = pd.read_csv('/content/drive/My Drive/Santander/train.csv')
test = pd.read_csv('/content/drive/My Drive/Santander/test.csv')
sub = pd.read_csv('/content/drive/My Drive/Santander/sample_submission.csv')


y = data['target']
del data['target']
data.drop(['Unnamed: 0','ID_code'], axis=1, inplace=True)
test.drop(['Unnamed: 0','ID_code'], axis=1, inplace=True)


# Modeling
folds = KFold(n_splits=5, shuffle=True, random_state=123)
oof_preds = np.zeros(data.shape[0])
sub_preds = np.zeros(test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(data)):
    trn_x, trn_y = data.iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = data.iloc[val_idx], y.iloc[val_idx]
    
    clf = LGBMClassifier(
        n_estimators=2000,
        learning_rate=0.1,
        num_leaves=123,
        colsample_bytree=.8,
        subsample=.9,
        max_depth=15,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2
    )
    
    clf.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric='auc', verbose=250, early_stopping_rounds=150
           )
    
    oof_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    sub_preds += clf.predict_proba(test, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
    del clf, trn_x, trn_y, val_x, val_y
    gc.collect()
    
print('Full AUC score %.6f' % roc_auc_score(y, oof_preds))   

sub['target'] = sub_preds

sub.to_csv('/content/drive/My Drive/Santander/submissionLight.csv', index=False, float_format='%.8f')

Training until validation scores don't improve for 150 rounds.
[250]	training's auc: 0.999838	training's binary_logloss: 0.0639669	valid_1's auc: 0.881791	valid_1's binary_logloss: 0.225034
Early stopping, best iteration is:
[233]	training's auc: 0.999682	training's binary_logloss: 0.0693493	valid_1's auc: 0.881377	valid_1's binary_logloss: 0.22466
Fold  1 AUC : 0.881377
Training until validation scores don't improve for 150 rounds.
[250]	training's auc: 0.999862	training's binary_logloss: 0.0648364	valid_1's auc: 0.886181	valid_1's binary_logloss: 0.217501
Early stopping, best iteration is:
[236]	training's auc: 0.999748	training's binary_logloss: 0.0692357	valid_1's auc: 0.886111	valid_1's binary_logloss: 0.217336
Fold  2 AUC : 0.886111
Training until validation scores don't improve for 150 rounds.
[250]	training's auc: 0.999876	training's binary_logloss: 0.0634816	valid_1's auc: 0.875854	valid_1's binary_logloss: 0.229124
Early stopping, best iteration is:
[234]	training's auc: 0.99